In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
from PIL import Image 
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

d:\anaconda3\envs\HF\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Prepare data

In [2]:

def get_sub_path(path):
    sub_path = []
    if isinstance(path, list):
        for p in path:
            if os.path.isdir(p):
                for file in os.listdir(p):
                    sub_path.append(os.path.join(p, file))
            else:
                continue
    else:
        for file in os.listdir(path):
            sub_path.append(os.path.join(path, file))
    return sub_path

def divide_list(list, n):
    for i in range(0, len(list), n):
        yield list[i:i + n]
        
def std(input):
    if input.max() == 0:
        return input
    else:
        result = (input-input.min()) / (input.max()-input.min())
        return result

In [3]:
feature_list = ['IR_drop_features_decompressed/power_i', 'IR_drop_features_decompressed/power_s', 
        'IR_drop_features_decompressed/power_sca', 'IR_drop_features_decompressed/power_all']
label_list = ['IR_drop_features_decompressed/IR_drop']
        
datapath = '../../CircuitNet/CircuitNet-N28/'
name_list = get_sub_path(os.path.join(datapath, feature_list[-1]))
n_list = divide_list(name_list, 1000)

In [4]:
class PowerDataset(Dataset):
    def __init__(self, root_dir, target_size=(224, 224)):
        self.root_dir = root_dir
        self.feature_dirs = ['power_i', 'power_s', 'power_sca', 'Power_all']
        self.label_dir = 'IR_drop'
        self.target_size = target_size
        # Collect all the feature and label file paths
        self.data = []
        i=0
        for case_name in os.listdir(os.path.join(root_dir, self.feature_dirs[0])):
            feature_paths = [os.path.join(root_dir, feature_dir, case_name) for feature_dir in self.feature_dirs]
            label_path = os.path.join(root_dir, self.label_dir, case_name)
            if all(os.path.exists(fp) for fp in feature_paths) and os.path.exists(label_path):
                self.data.append((feature_paths, label_path))
            i+=1
            if i>100:
                break

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        feature_paths, label_path = self.data[idx]
        features = []
        
        for fp in feature_paths:
            feature = np.load(fp)
            feature = torch.tensor(feature, dtype=torch.float32)
            feature = F.interpolate(feature.unsqueeze(0).unsqueeze(0), size=self.target_size, mode='nearest').squeeze(0).squeeze(0)
            feature = std(feature)
            features.append(feature)
            
        features = torch.stack(features, dim=0)
        
        # Load and process label file
        label = np.load(label_path)
        label = torch.tensor(label, dtype=torch.float32)
        label = F.interpolate(label.unsqueeze(0).unsqueeze(0), size=self.target_size, mode='nearest').squeeze(0).squeeze(0)
        label = label.clamp(1e-6, 50)
        label = (torch.log10(label)+6)/ (np.log10(50)+6)
        
        return features, label
    
root_dir = '../../CircuitNet/CircuitNet-N28/IR_drop_features_decompressed/'
dataset = PowerDataset(root_dir)

In [5]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

for features, labels in dataloader:
    print(features.shape, labels.shape)
    break

torch.Size([2, 4, 224, 224]) torch.Size([2, 224, 224])


In [6]:
from swintransformer import *

model_name = 'swin_base_patch4_window7_224'
model = init_model(model_name, input_channels=4, num_classes=0, pretrained=True)

In [7]:
model.eval()
with torch.no_grad():
    ir_prediction = model(features)

In [8]:
ir_prediction.shape

torch.Size([2, 1, 224, 224])